# Step 3 - Prepare points of interest (POI) data
## Project: Growing Urban Bicycle Networks - with an LTN twist

This notebook downloads and prepares points of interest data to be used for bicycle network design, snapping them to the networks created in 02_prepare_networks. POIs (also called seed points) can be set using: 
* LTNs
* Hexagonal Tesselations within city boundary and external to LTNs
* railway stations
* grid points

## Preliminaries

### Parameters

In [ ]:
debug = True # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"

### Setup

In [ ]:
%run -i path.py
%run -i setup.py

%load_ext watermark
#%watermark -n -v -m -g -iv
PATH['exports_gpkg'] = '../../bikenwgrowth_external/exports_gpkg/'

### Functions

In [ ]:
%run -i functions.py

## Download and wrangle data

In [ ]:
scenarios = ["no_ltn_scenario", "current_ltn_scenario", "more_ltn_scenario"]

# Load all carall graphs in OSMNX format
G_caralls = {}
G_caralls_simplified = {}
G_biketracks = {}
locations = {}
parameterinfo = osmnxparameters['carall']

for scenario in scenarios:
    for placeid, placeinfo in tqdm(cities.items(), desc=f"Cities ({scenario})"):
        print(placeid + ": Loading location polygon and carall graph")
        
        if placeinfo["nominatimstring"] != '':
            location = ox.geocoder.geocode_to_gdf(placeinfo["nominatimstring"])
            if location.geometry[0].geom_type == 'MultiPolygon':
                location = location.explode(index_parts=False).reset_index(drop=True)
            location = fill_holes(extract_relevant_polygon(placeid, shapely.geometry.shape(location['geometry'][0])))
        else:
            # https://gis.stackexchange.com/questions/113799/how-to-read-a-shapefile-in-python
            shp = fiona.open(PATH["data"] + placeid + "/" + placeid + ".shp")
            first = next(iter(shp))
            try:
                location = Polygon(shapely.geometry.shape(first['geometry'])) # If shape file is given as linestring
            except:
                location = shapely.geometry.shape(first['geometry'])
        locations[(placeid, scenario)] = location
        
        base_path = os.path.join(PATH["data"], placeid, scenario) + "/"
        G_caralls[(placeid, scenario)] = csv_to_ox(base_path, placeid, 'carall')
        G_caralls[(placeid, scenario)].graph["crs"] = 'epsg:4326' # needed for OSMNX's graph_to_gdfs in utils_graph.py
        G_caralls_simplified[(placeid, scenario)] = csv_to_ox(base_path, placeid, 'carall_simplified')
        G_caralls_simplified[(placeid, scenario)].graph["crs"] = 'epsg:4326' # needed for OSMNX's graph_to_gdfs in utils_graph.py

        G_biketracks[(placeid, scenario)] = csv_to_ox(base_path, placeid, 'biketrack')
        G_biketracks[(placeid, scenario)].graph["crs"] = 'epsg:4326'

ltn_nnids_dict  = {}
tess_nnids_dict = {}
LTN_gdf_dict = {}
snapped_gdf_dict = {}


### Tessellation

In [ ]:
# get POIs based on hexagonal tessellation within the city boundary whilst excluding LTNs
for scenario in scenarios:
    for placeid, placeinfo in tqdm(cities.items(), desc=f"Cities ({scenario})"):
        # skip tessellation if there are no LTNs
        if scenario == "no_ltn_scenario":
            continue

        print(f"{placeid}: Tessellation around neighbourhoods ({scenario})")
        location = locations[(placeid, scenario)]

        location_gseries = gpd.GeoSeries([location])
        location_gdf = gpd.GeoDataFrame(geometry=location_gseries)
        location_gdf.set_crs(epsg=4326, inplace=True)

        # Get urban areas for seed points
        urban_areas = get_urban_areas(placeinfo["nominatimstring"])
        urban_areas = urban_areas.clip(location_gdf.unary_union)

        tess = Tessellation(urban_areas)
        tess.get_polygon()  # Optional plot for debugging
        hexagons = tess.hexagons(resolution=8)
        clean_hexagons = hexagons  # save a copy for later!
        if export:
            file_path = os.path.join(PATH["exports_gpkg"], placeid, scenario, f"{placeid}_clean_hexagons.gpkg")
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            hexagons.to_file(file_path, driver="GPKG")

        # Load LTNs
        neighbourhoods = load_neighbourhoods(os.path.join(PATH["data"], placeid, scenario, ""))
        neighbourhood_geoms = [
            geom for gdf in neighbourhoods.values() for geom in gdf['geometry'].values
        ]
        neighbourhoods_gdf = MultiPolygon(neighbourhood_geoms)
        for neighbourhood_geom in neighbourhoods_gdf.geoms:
            location = location.difference(neighbourhood_geom)

        # Set updated location as a GeoDataFrame
        location_gdf = gpd.GeoDataFrame(geometry=[location], crs="EPSG:4326")

        # Remove hexagons that intersect with neighbourhoods
        hexagons = hexagons[~hexagons.intersects(neighbourhoods_gdf)]
        hexagon_centroids_gdf = gpd.GeoDataFrame(geometry=hexagons.centroid, crs=hexagons.crs)
        if export:
            file_path = os.path.join(PATH["exports_gpkg"], placeid, scenario, f"{placeid}_hexagons.gpkg")
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            hexagons.to_file(file_path, driver="GPKG")
            file_path = os.path.join(PATH["exports_gpkg"], placeid, scenario, f"{placeid}_hexagon_centroids.gpkg")
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            hexagon_centroids_gdf.to_file(file_path, driver="GPKG")

        if debug:
            hexagon_centroids_gdf.plot(color='blue')

        print(placeid + ": loading and moving POIs")
        # load networks
        G_carall    = G_caralls_simplified[(placeid, scenario)]
        G_biketrack = G_biketracks[(placeid, scenario)]

        # First snap points to the carall network
        updated_points = []
        for g in hexagon_centroids_gdf['geometry']:
            n = ox.distance.nearest_nodes(G_carall, g.x, g.y)
            distance_to_nearest_node = haversine(
                (g.y, g.x),
                (G_carall.nodes[n]["y"], G_carall.nodes[n]["x"]),
                unit="m"
            )
            if distance_to_nearest_node <= snapthreshold:
                snapped_point = Point(G_carall.nodes[n]["x"], G_carall.nodes[n]["y"])
                updated_points.append({'geometry': snapped_point, 'snap_status': 'snapped'})
            else:
                updated_points.append({'geometry': g, 'snap_status': 'unsnapped'})

        snapped_gdf    = gpd.GeoDataFrame(updated_points, crs=hexagon_centroids_gdf.crs)
        snapped_carall = snapped_gdf[snapped_gdf['snap_status'] != 'unsnapped']
        if export:
            file_path = os.path.join(PATH["exports_gpkg"], placeid, scenario, f"{placeid}_hexagon_centroids_snapped_unsnapped.gpkg")
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            snapped_gdf.to_file(file_path, driver="GPKG")

        if debug:
            fig, ax = plt.subplots(figsize=(10, 10))
            location_gdf.plot(ax=ax)
            urban_areas.plot(ax=ax, color='red')
            hexagons.plot(ax=ax, color="lightgray", edgecolor="white", alpha=0.5, label="Hexagons")
            snapped_gdf[snapped_gdf['snap_status'] == 'snapped'].plot(ax=ax, color='green', label='Snapped to Carall')
            snapped_gdf[snapped_gdf['snap_status'] == 'unsnapped'].plot(ax=ax, color='red', label='Unsnapped')
            if isinstance(neighbourhoods_gdf, MultiPolygon):
                neighbourhoods_gdf_for_plotting = gpd.GeoDataFrame(geometry=[neighbourhoods_gdf], crs="EPSG:4326")
                neighbourhoods_gdf_for_plotting = neighbourhoods_gdf_for_plotting.explode()
                neighbourhoods_gdf_for_plotting.plot(ax=ax, color='green', alpha=0.5, label='LTNs')

            plt.legend()
            plt.title("Snapped and Unsnapped Hexagon Centroids to Street Network (removing points which don't snap inside the boundary)")
            plt.show()

        print(placeid + ": Snapping to bike tracks")
        # Identify common nodes between G_biketrack and G_carall
        bike_nodes   = set(G_biketrack.nodes)
        car_nodes    = set(G_carall.nodes)
        common_nodes = bike_nodes & car_nodes

        common_node_coords = {
            n: (G_carall.nodes[n]["x"], G_carall.nodes[n]["y"])
            for n in common_nodes
        }

        # Snap snapped_carall points to bike track network
        updated_bike_points = []
        for g in snapped_carall['geometry']:
            nearest_node, min_distance = None, float('inf')
            for n, (x, y) in common_node_coords.items():
                distance = haversine((g.y, g.x), (y, x), unit="m")
                if distance < min_distance:
                    nearest_node, min_distance = n, distance
            if nearest_node is not None and min_distance <= snapthreshold:
                snapped_bike_point = Point(common_node_coords[nearest_node])
                updated_bike_points.append({'geometry': snapped_bike_point, 'snap_status': 'snapped'})
            else:
                updated_bike_points.append({'geometry': g, 'snap_status': 'unsnapped'})

        snapped_biketrack = gpd.GeoDataFrame(updated_bike_points, crs=hexagon_centroids_gdf.crs)
        if debug:
            fig, ax = plt.subplots(figsize=(10, 10))
            hexagons.plot(ax=ax, color="lightgray", edgecolor="white", alpha=0.5, label="Hexagons")
            snapped_biketrack[snapped_biketrack['snap_status'] == 'snapped'].plot(ax=ax, color='green', label='Snapped to Bike Track')
            snapped_biketrack[snapped_biketrack['snap_status'] == 'unsnapped'].plot(ax=ax, color='red', label='Unsnapped')
            plt.legend()
            plt.title(f"Snapped and Unsnapped Hexagon Centroids to Bike Track for {placeid}")
            plt.show()
        if export:
            file_path = os.path.join(PATH["exports_gpkg"], placeid, scenario, f"{placeid}_hexagon_centroids_bike_snapped_unsnapped.gpkg")
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            snapped_biketrack.to_file(file_path, driver="GPKG")

        # For any unsnapped points, snap to stroke
        nodes, edges = ox.graph_to_gdfs(G_caralls[(placeid, scenario)].to_undirected())
        coins       = momepy.COINS(edges)
        stroke_gdf  = coins.stroke_gdf().to_crs(epsg=4326)
        if export:
            file_path = os.path.join(PATH["exports_gpkg"], placeid, scenario, f"{placeid}_strokes.gpkg")
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            stroke_gdf.to_file(file_path, driver="GPKG")

        unsnapped = snapped_biketrack[snapped_biketrack['snap_status'] == 'unsnapped'].copy()
        unsnapped = unsnapped.to_crs(epsg=3857)
        stroke_sindex = stroke_gdf.sindex
        unsnapped["geometry"] = unsnapped["geometry"].apply(
            lambda p: snap_to_largest_stroke(p, snapthreshold, stroke_gdf)
        )
        unsnapped["snap_status"] = "snapped"
        unsnapped = unsnapped.to_crs(epsg=4326)

        snapped_biketrack = snapped_biketrack[snapped_biketrack['snap_status'] != 'unsnapped']
        snapped_biketrack = pd.concat([snapped_biketrack, unsnapped], ignore_index=True)
        snapped_gdf_dict[(placeid, scenario)] = snapped_biketrack

        # Save nearest node IDs
        nnids = set()
        for g in snapped_biketrack['geometry']:
            nearest_node, min_distance = None, float('inf')
            for n, (x, y) in common_node_coords.items():
                distance = haversine((g.y, g.x), (y, x), unit="m")
                if distance < min_distance:
                    nearest_node, min_distance = n, distance
            if nearest_node is not None:
                nnids.add(nearest_node)
        tess_nnids_dict[(placeid, scenario)] = nnids
        nnids_csv_path = os.path.join(PATH["data"], placeid, scenario, f"{placeid}_poi_tessellation_nnidsbikeall.csv")
        with open(nnids_csv_path, 'w') as f:
            for item in nnids:
                f.write(f"{item}\n")

        gpkg_output_path = os.path.join(PATH["data"], placeid, scenario, f"{placeid}_poi_tessellation.gpkg")
        try:
            snapped_biketrack.to_file(gpkg_output_path, driver='GPKG')
        except Exception as e:
            print(f"Notice: Writing the tessellation gdf did not work for {placeid}. Error: {e}")

         # reload neighbourhoods & LTN MultiPolygon for this scenario
        neighbourhoods = load_neighbourhoods(os.path.join(PATH["data"], placeid, scenario, ""))
        neighbourhood_geoms = [geom for gdf in neighbourhoods.values() for geom in gdf['geometry'].values]
        neighbourhoods_gdf = MultiPolygon(neighbourhood_geoms)

        if debug:
            # convert MultiPolygon → exploded GeoDataFrame
            if isinstance(neighbourhoods_gdf, MultiPolygon):
                neighbourhoods_gdf = gpd.GeoDataFrame(geometry=[neighbourhoods_gdf], crs="EPSG:4326")
                neighbourhoods_gdf = neighbourhoods_gdf.explode()

            # centroids of each LTN polygon
            neighbourhood_centroids = neighbourhoods_gdf.copy()
            neighbourhood_centroids['geometry'] = neighbourhood_centroids.geometry.centroid

            # now plot everything
            fig, ax = plt.subplots(figsize=(10, 10))
            # city boundary (with LTN holes already subtracted)
            location_gdf.plot(ax=ax, color='lightgrey', label='City boundary')
            # urban areas
            urban_areas.plot(ax=ax, color='orange', alpha=0.6, label='Urban Areas')
            # LTN polygons
            neighbourhoods_gdf.plot(ax=ax, color='green', label='LTNs')
            # neighbourhood seed points
            neighbourhood_centroids.plot(ax=ax, color='red', label='LTN centroids')
            # hexagon grid
            hexagons.plot(ax=ax, color="lightgray", edgecolor="white", alpha=0.5, label="Hexagons")
            # snapped bike seed points
            snapped_biketrack[snapped_biketrack['snap_status'] == 'snapped'] \
                .plot(ax=ax, color='green', label='Tessellation seed points')
            
            ax.axis('off')
            plt.legend()
            plt.show()

            
    if methods_plotting:
        # skip this if we're on the no_ltn_scenario
        if scenario == "no_ltn_scenario":
            continue

        # plot city
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.axis('off')
        location = locations[(placeid, scenario)]
        location_gseries = gpd.GeoSeries([location])
        location_gdf = gpd.GeoDataFrame(geometry=location_gseries)
        location_gdf = location_gdf.set_crs(epsg=4326, inplace=True)
        location_gdf.plot(ax=ax, color='lightgrey')
        plt.savefig(
            os.path.join(PATH["plots"], placeid, scenario, "city_boundary.png"),
            dpi=600,
            bbox_inches='tight'
        )

        # plot city with roads on
        road_network = G_caralls[(placeid, scenario)]
        road_network_gdf = ox.graph_to_gdfs(road_network, nodes=False)
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.axis('off')
        location_gdf.plot(ax=ax, color='lightgrey')
        road_network_gdf.plot(ax=ax, color='black', alpha=0.5, linewidth=0.5)
        plt.savefig(
            os.path.join(PATH["plots"], placeid, scenario, "city_boundary_with_roads.png"),
            dpi=600,
            bbox_inches='tight'
        )

        # plot urban areas
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.axis('off')
        location_gdf.plot(ax=ax, color='lightgrey')
        urban_areas = get_urban_areas(placeinfo["nominatimstring"])
        urban_areas = urban_areas.clip(location_gdf.geometry.iloc[0])
        urban_areas.plot(ax=ax, color='orange', alpha=0.6, label='Urban Areas')
        road_network_gdf.plot(ax=ax, color='black', alpha=0.5, linewidth=0.5, zorder=1)
        plt.savefig(
            os.path.join(PATH["plots"], placeid, scenario, "urban_areas.png"),
            dpi=600,
            bbox_inches='tight'
        )

        # plot neighbourhoods
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.axis('off')
        location_gdf.plot(ax=ax, color='lightgrey', zorder=0)

        neighbourhoods = load_neighbourhoods(os.path.join(PATH["data"], placeid, scenario, ""))
        geoms = [g for gdf in neighbourhoods.values() for g in gdf.geometry]
        neighbourhoods_gdf = gpd.GeoDataFrame(geometry=[MultiPolygon(geoms)], crs="EPSG:4326").explode()

        neighbourhoods_gdf.plot(ax=ax, color='green', label='LTNs', alpha=0.8, zorder=2)
        road_network_gdf.plot(ax=ax, color='black', alpha=0.5, linewidth=0.5, zorder=1)
        plt.savefig(
            os.path.join(PATH["plots"], placeid, scenario, "LTNs.png"),
            dpi=600,
            bbox_inches='tight'
        )

        # plot hexagons
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.axis('off')
        location_gdf.plot(ax=ax, color='lightgrey', zorder=0)
        urban_areas.plot(ax=ax, color='orange', alpha=0.6, label='Urban Areas', zorder=2)
        clean_hexagons.plot(ax=ax, color="lightblue", edgecolor="white", alpha=0.8, zorder=3)
        road_network_gdf.plot(ax=ax, color='black', alpha=0.5, linewidth=0.5, zorder=1)
        plt.savefig(
            os.path.join(PATH["plots"], placeid, scenario, "hexagons.png"),
            dpi=600,
            bbox_inches='tight'
        )

        # plot hexagons around LTNs
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.axis('off')
        location_gdf.plot(ax=ax, color='lightgrey', zorder=0)
        neighbourhoods_gdf.plot(ax=ax, color='green', label='LTNs', alpha=0.8, zorder=2)
        hexagons.plot(ax=ax, color="lightblue", edgecolor="white", alpha=0.8, zorder=3)
        road_network_gdf.plot(ax=ax, color='black', alpha=0.5, linewidth=0.5, zorder=1)
        plt.savefig(
            os.path.join(PATH["plots"], placeid, scenario, "hexagons_around_LTNs.png"),
            dpi=600,
            bbox_inches='tight'
        )

        # plot centroids
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.axis('off')
        location_gdf.plot(ax=ax, color='lightgrey', zorder=0)
        neighbourhoods_gdf.plot(ax=ax, color='green', label='LTNs', alpha=0.8, zorder=2)
        hexagons.plot(ax=ax, color="lightblue", edgecolor="white", alpha=0.8, zorder=3)
        snapped_biketrack.plot(ax=ax, color='red', label='Snapped Points', zorder=4)

        neighbourhood_centroids = neighbourhoods_gdf.copy()
        neighbourhood_centroids['geometry'] = neighbourhood_centroids.geometry.centroid
        neighbourhood_centroids.plot(ax=ax, color='blue', label='Neighbourhood Centroids', zorder=5)

        road_network_gdf.plot(ax=ax, color='black', alpha=0.5, linewidth=0.5, zorder=1)
        plt.savefig(
            os.path.join(PATH["plots"], placeid, scenario, "centroids.png"),
            dpi=600,
            bbox_inches='tight'
        )

        base_path = os.path.join(PATH["plots"], placeid, scenario)
        image_files = [
            "urban_areas.png",
            "hexagons.png",
            "hexagons_around_LTNs.png",
            "centroids.png"
        ]
        titles = ["Define urban area", "Generate hexagons in urban areas", "Remove hexagons within LTNs", "Create and snap seed points"]
        image_paths = [os.path.join(base_path, img) for img in image_files]
        fig, axs = plt.subplots(1, 4, figsize=(20, 5))
        for ax, img_path, title in zip(axs, image_paths, titles):
            img = mpimg.imread(img_path)
            ax.imshow(img)
            ax.axis('off')
            ax.set_title(title, fontsize=10)

        combined_path = os.path.join(base_path, "seed_point_methods_workflow.png")
        plt.tight_layout()
        plt.savefig(combined_path, dpi=600, bbox_inches='tight')
        plt.close()

        print("Plots saved to: " + os.path.join(PATH["plots"], placeid, scenario) + "/")




### Low Traffic Neighbourhood Centroids

In [ ]:


for scenario in scenarios:
    for placeid, placeinfo in tqdm(cities.items(), desc=f"Cities ({scenario})"):
        # skip this if we're on the no_ltn_scenario
        if scenario == "no_ltn_scenario":
            continue

        print(placeid + ": loading and moving LTNs")
        
        # Load LTN POIs
        if placeinfo["nominatimstring"] != '':
            location = ox.geocoder.geocode_to_gdf(placeinfo["nominatimstring"])
            if location.geometry[0].geom_type == 'MultiPolygon':
                location = location.explode(index_parts=False).reset_index(drop=True)
            location = fill_holes(extract_relevant_polygon(placeid, shapely.geometry.shape(location['geometry'][0])))
        else:
            # https://gis.stackexchange.com/questions/113799/how-to-read-a-shapefile-in-python
            shp = fiona.open(PATH["data"] + placeid + "/" + scenario + "/" + placeid + ".shp")
            first = next(iter(shp))
            try:
                location = Polygon(shapely.geometry.shape(first['geometry']))  # If shape file is given as linestring
            except:
                location = shapely.geometry.shape(first['geometry'])
        locations[(placeid, scenario)] = location
        
        G_caralls[(placeid, scenario)] = csv_to_ox_highway(
            PATH["data"] + placeid + "/" + scenario + "/", placeid, 'carall'
        )
        G_caralls[(placeid, scenario)].graph["crs"] = 'epsg:4326'  # Needed for OSMNX's graph_to_gdfs in utils_graph.py
        G_caralls_simplified[(placeid, scenario)] = csv_to_ox(
            PATH["data"] + placeid + "/" + scenario + "/", placeid, 'carall_simplified'
        )
        G_caralls_simplified[(placeid, scenario)].graph["crs"] = 'epsg:4326'  # Needed for OSMNX's graph_to_gdfs in utils_graph.py

        print(f"{placeid}: Loading and moving POIs")
        # Get the carall graph and location geometry
        location = locations[(placeid, scenario)]
        G_carall = G_caralls_simplified[(placeid, scenario)]

        # Load neighbourhoods and create GeoDataFrame for centroids
        neighbourhoods = load_neighbourhoods(PATH["data"] + placeid + "/" + scenario + "/")
        LTNs = gpd.GeoDataFrame(columns=['neighbourhood_id', 'geometry'], crs='EPSG:4326')  
        
        unique_id = 0 
        for name, gdf in neighbourhoods.items():  # Process each neighbourhood GeoDataFrame to get centroids, exit points, and neighbourhood IDs
            if gdf.empty:
                print(f"Warning: The GeoDataFrame for {name} is empty. Skipping...")
                continue
            print(f"Processing neighbourhoods in: {name}")

            # Assign a unique ID to each neighbourhood in the GeoDataFrame to reference throughout
            gdf['neighbourhood_id'] = range(unique_id, unique_id + len(gdf))
            if debug:
                print(f"Assigned neighbourhood_ids from {unique_id} to {unique_id + len(gdf) - 1} for {name}")

            # Get centroids to inherit 'neighbourhood_id'
            centroids_gdf = get_neighbourhood_centroids(gdf)
            LTNs = pd.concat([LTNs, centroids_gdf], ignore_index=True)
            unique_id += len(gdf)  # Increment by the number of neighbourhoods processed

        # Snap centroids to the closest nodes in the street network
        neighbourhood_nnids = set()
        for g in LTNs['geometry']:
            n = ox.distance.nearest_nodes(G_carall, g.x, g.y)
            if n not in neighbourhood_nnids and haversine(
                (g.y, g.x),
                (G_carall.nodes[n]["y"], G_carall.nodes[n]["x"]),
                unit="m"
            ) <= snapthreshold:
                neighbourhood_nnids.add(n)
        # Add nearest_node column to LTNs by finding the nearest node for each centroid geometry
        LTNs['nearest_node'] = LTNs['geometry'].apply(
            lambda g: ox.distance.nearest_nodes(G_carall, g.x, g.y)
        )  # We now have LTNs with 'neighbourhood_id', 'geometry', 'nearest_node' columns

        ltn_nnids = get_nearest_nodes_to_gdf(G_carall, LTNs)
        ltn_nnids_dict[(placeid, scenario)] = ltn_nnids
        LTN_gdf_dict[(placeid, scenario)] = LTNs
        snapped_gdf_dict[(placeid, scenario)] = snapped_biketrack

        # save to file
        nnids_csv_path = PATH["data"] + placeid + "/" + scenario + "/" + placeid + '_poi_LTNs_nnids.csv'
        with open(nnids_csv_path, 'w') as f:
            for item in ltn_nnids:
                f.write("%s\n" % item)
        gpkg_output_path = PATH["data"] + placeid + "/" + scenario + "/" + placeid + '_poi_LTNs.gpkg'
        try:
            LTNs.to_file(gpkg_output_path, driver='GPKG')
        except Exception as e:
            print(f"Notice: Writing the LTN gdf did not work for {placeid}. Error: {e}")


### Combine LTNs and Tesselation

In [ ]:
for scenario in scenarios:
    for placeid, placeinfo in tqdm(cities.items(), desc=f"Combine NNIDs ({scenario})"):
        if scenario == "no_ltn_scenario":
            continue

        # fetch the two lists
        ltn_nnids         = ltn_nnids_dict[(placeid, scenario)]
        tess_nnids        = tess_nnids_dict[(placeid, scenario)]
        LTN_gdf           = LTN_gdf_dict[(placeid, scenario)]
        snapped_biketrack = snapped_gdf_dict[(placeid, scenario)]

        # combine LTN + tessellation NNIDs
        ltn_and_tess_nnids = ltn_nnids + list(tess_nnids)
        nnids_csv_path = os.path.join(
            PATH["data"], placeid, scenario,
            f"{placeid}_poi_LTNs_tessellation_nnidscarall.csv"
        )
        with open(nnids_csv_path, 'w') as f:
            for item in ltn_and_tess_nnids:
                f.write(f"{item}\n")

        # combine the two GeoDataFrames into one
        combined_gdf = pd.concat([LTN_gdf, snapped_biketrack], ignore_index=True)
        combined_gpkg_output_path = os.path.join(
            PATH["data"], placeid, scenario,
            f"{placeid}_poi_LTNs_tessellation.gpkg"
        )
        try:
            combined_gdf.to_file(combined_gpkg_output_path, driver='GPKG')
        except Exception as e:
            print(f"Notice: Writing the combined gdf did not work for {placeid}. Error: {e}")


## No LTNs scenario

In [ ]:
for scenario in scenarios:
    if scenario == "no_ltn_scenario":
        for placeid, placeinfo in tqdm(cities.items(), desc=f"Cities ({scenario})"):

            print(f"{placeid}: Tessellation (NO LTNs) — {scenario}")
            location = locations[(placeid, scenario)]

            location_gseries = gpd.GeoSeries([location])
            location_gdf = gpd.GeoDataFrame(geometry=location_gseries)
            location_gdf.set_crs(epsg=4326, inplace=True)

            # Get urban areas for seed points
            urban_areas = get_urban_areas(placeinfo["nominatimstring"])
            urban_areas = urban_areas.clip(location_gdf.unary_union)

            tess = Tessellation(urban_areas)
            tess.get_polygon()
            hexagons = tess.hexagons(resolution=8)
            clean_hexagons = hexagons  # save a copy for later

            if export:
                file_path = os.path.join(PATH["exports_gpkg"], placeid, scenario, f"{placeid}_clean_hexagons.gpkg")
                os.makedirs(os.path.dirname(file_path), exist_ok=True)
                hexagons.to_file(file_path, driver="GPKG")

            hexagon_centroids_gdf = gpd.GeoDataFrame(geometry=hexagons.centroid, crs=hexagons.crs)

            print(placeid + ": snapping to carall network")
            G_carall = G_caralls_simplified[(placeid, scenario)]
            updated_points = []
            for g in hexagon_centroids_gdf['geometry']:
                n = ox.distance.nearest_nodes(G_carall, g.x, g.y)
                distance_to_nearest_node = haversine(
                    (g.y, g.x),
                    (G_carall.nodes[n]["y"], G_carall.nodes[n]["x"]),
                    unit="m"
                )
                if distance_to_nearest_node <= snapthreshold:
                    snapped_point = Point(G_carall.nodes[n]["x"], G_carall.nodes[n]["y"])
                    updated_points.append({'geometry': snapped_point, 'snap_status': 'snapped'})
                else:
                    updated_points.append({'geometry': g, 'snap_status': 'unsnapped'})

            snapped_gdf = gpd.GeoDataFrame(updated_points, crs=hexagon_centroids_gdf.crs)
            snapped_carall = snapped_gdf[snapped_gdf['snap_status'] != 'unsnapped']

            if export:
                file_path = os.path.join(PATH["exports_gpkg"], placeid, scenario, f"{placeid}_hexagon_centroids_snapped_unsnapped.gpkg")
                os.makedirs(os.path.dirname(file_path), exist_ok=True)
                snapped_gdf.to_file(file_path, driver="GPKG")

            print(placeid + ": snapping to bike track")
            G_biketrack = G_biketracks[(placeid, scenario)]
            bike_nodes = set(G_biketrack.nodes)
            car_nodes = set(G_carall.nodes)
            common_nodes = bike_nodes & car_nodes
            common_node_coords = {
                n: (G_carall.nodes[n]["x"], G_carall.nodes[n]["y"])
                for n in common_nodes
            }

            updated_bike_points = []
            for g in snapped_carall['geometry']:
                nearest_node, min_distance = None, float('inf')
                for n, (x, y) in common_node_coords.items():
                    distance = haversine((g.y, g.x), (y, x), unit="m")
                    if distance < min_distance:
                        nearest_node, min_distance = n, distance
                if nearest_node is not None and min_distance <= snapthreshold:
                    snapped_bike_point = Point(common_node_coords[nearest_node])
                    updated_bike_points.append({'geometry': snapped_bike_point, 'snap_status': 'snapped'})
                else:
                    updated_bike_points.append({'geometry': g, 'snap_status': 'unsnapped'})

            snapped_biketrack = gpd.GeoDataFrame(updated_bike_points, crs=hexagon_centroids_gdf.crs)

            print(placeid + ": snapping remaining to strokes")
            nodes, edges = ox.graph_to_gdfs(G_caralls[(placeid, scenario)].to_undirected())
            coins = momepy.COINS(edges)
            stroke_gdf = coins.stroke_gdf().to_crs(epsg=4326)
            if export:
                file_path = os.path.join(PATH["exports_gpkg"], placeid, scenario, f"{placeid}_strokes.gpkg")
                os.makedirs(os.path.dirname(file_path), exist_ok=True)
                stroke_gdf.to_file(file_path, driver="GPKG")

            unsnapped = snapped_biketrack[snapped_biketrack['snap_status'] == 'unsnapped'].copy()
            unsnapped = unsnapped.to_crs(epsg=3857)
            stroke_sindex = stroke_gdf.sindex
            unsnapped["geometry"] = unsnapped["geometry"].apply(
                lambda p: snap_to_largest_stroke(p, snapthreshold, stroke_gdf)
            )
            unsnapped["snap_status"] = "snapped"
            unsnapped = unsnapped.to_crs(epsg=4326)

            snapped_biketrack = snapped_biketrack[snapped_biketrack['snap_status'] != 'unsnapped']
            snapped_biketrack = pd.concat([snapped_biketrack, unsnapped], ignore_index=True)
            snapped_gdf_dict[(placeid, scenario)] = snapped_biketrack

            # Save nearest node IDs
            nnids = set()
            for g in snapped_biketrack['geometry']:
                nearest_node, min_distance = None, float('inf')
                for n, (x, y) in common_node_coords.items():
                    distance = haversine((g.y, g.x), (y, x), unit="m")
                    if distance < min_distance:
                        nearest_node, min_distance = n, distance
                if nearest_node is not None:
                    nnids.add(nearest_node)

            tess_nnids_dict[(placeid, scenario)] = nnids
            nnids_csv_path = os.path.join(PATH["data"], placeid, scenario, f"{placeid}_poi_tessellation_nnidsbikeall.csv")
            with open(nnids_csv_path, 'w') as f:
                for item in nnids:
                    f.write(f"{item}\n")

            gpkg_output_path = os.path.join(PATH["data"], placeid, scenario, f"{placeid}_poi_tessellation.gpkg")
            try:
                snapped_biketrack.to_file(gpkg_output_path, driver='GPKG')
            except Exception as e:
                print(f"Notice: Writing the tessellation GDF failed for {placeid}. Error: {e}")


In [ ]:
if methods_plotting:
    for scenario in scenarios:
        if scenario == "no_ltn_scenario":
            for placeid, placeinfo in tqdm(cities.items(), desc=f"Cities ({scenario})"):
                #  Plot city boundary
                location = locations[(placeid, scenario)]
                location_gseries = gpd.GeoSeries([location])
                location_gdf = gpd.GeoDataFrame(geometry=location_gseries, crs="EPSG:4326")

                fig, ax = plt.subplots(figsize=(10, 10))
                ax.axis('off')
                location_gdf.plot(ax=ax, color='lightgrey')
                plt.savefig(
                    os.path.join(PATH["plots"], placeid, scenario, "city_boundary.png"),
                    dpi=600, bbox_inches='tight'
                )

                # Plot road network over city boundary
                road_network = G_caralls[(placeid, scenario)]
                road_network_gdf = ox.graph_to_gdfs(road_network, nodes=False)

                fig, ax = plt.subplots(figsize=(10, 10))
                ax.axis('off')
                location_gdf.plot(ax=ax, color='lightgrey')
                road_network_gdf.plot(ax=ax, color='black', alpha=0.5, linewidth=0.5)
                plt.savefig(
                    os.path.join(PATH["plots"], placeid, scenario, "city_boundary_with_roads.png"),
                    dpi=600, bbox_inches='tight'
                )

                # Plot urban areas
                urban_areas = get_urban_areas(placeinfo["nominatimstring"])
                urban_areas = urban_areas.clip(location_gdf.geometry.iloc[0])

                fig, ax = plt.subplots(figsize=(10, 10))
                ax.axis('off')
                location_gdf.plot(ax=ax, color='lightgrey')
                urban_areas.plot(ax=ax, color='orange', alpha=0.6, label='Urban Areas')
                road_network_gdf.plot(ax=ax, color='black', alpha=0.5, linewidth=0.5)
                plt.savefig(
                    os.path.join(PATH["plots"], placeid, scenario, "urban_areas.png"),
                    dpi=600, bbox_inches='tight'
                )

                # Plot tessellation hexagons
                clean_hexagons = gpd.read_file(os.path.join(
                    PATH["exports_gpkg"], placeid, scenario, f"{placeid}_clean_hexagons.gpkg"))

                fig, ax = plt.subplots(figsize=(10, 10))
                ax.axis('off')
                location_gdf.plot(ax=ax, color='lightgrey')
                urban_areas.plot(ax=ax, color='orange', alpha=0.6)
                clean_hexagons.plot(ax=ax, color="lightblue", edgecolor="white", alpha=0.8)
                road_network_gdf.plot(ax=ax, color='black', alpha=0.5, linewidth=0.5)
                plt.savefig(
                    os.path.join(PATH["plots"], placeid, scenario, "hexagons.png"),
                    dpi=600, bbox_inches='tight'
                )

                # Plot snapped tessellation centroids (seed points)
                snapped_biketrack = gpd.read_file(os.path.join(
                    PATH["data"], placeid, scenario, f"{placeid}_poi_tessellation.gpkg"))

                fig, ax = plt.subplots(figsize=(10, 10))
                ax.axis('off')
                location_gdf.plot(ax=ax, color='lightgrey', zorder=0)
                clean_hexagons.plot(ax=ax, color="lightblue", edgecolor="white", alpha=0.8, zorder=1)
                road_network_gdf.plot(ax=ax, color='black', alpha=0.5, linewidth=0.5, zorder=2)
                snapped_biketrack.plot(ax=ax, color='red', label='Snapped Points', zorder=3)
                plt.savefig(
                    os.path.join(PATH["plots"], placeid, scenario, "centroids.png"),
                    dpi=600, bbox_inches='tight'
                )

                # Combine key steps into workflow image
                base_path = os.path.join(PATH["plots"], placeid, scenario)
                image_files = [
                    "urban_areas.png",
                    "hexagons.png",
                    "centroids.png"
                ]
                titles = [
                    "Define urban area",
                    "Generate hexagons in urban areas",
                    "Create and snap seed points"
                ]
                image_paths = [os.path.join(base_path, img) for img in image_files]
                fig, axs = plt.subplots(1, 3, figsize=(18, 5))
                for ax, img_path, title in zip(axs, image_paths, titles):
                    img = mpimg.imread(img_path)
                    ax.imshow(img)
                    ax.axis('off')
                    ax.set_title(title, fontsize=10)
                plt.tight_layout()
                plt.savefig(os.path.join(base_path, "seed_point_methods_workflow.png"), dpi=600, bbox_inches='tight')
                plt.close()

                print("Plots saved to: " + base_path)


### Neighbourhood centroids

In [ ]:
""" for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):
    print(placeid + ": loading and moving POIs")

    # We need the carall graph and location geometry
    location = locations[placeid]
    G_carall = G_caralls_simplified[placeid]

    # Load POIs from your local GeoPackage instead of OpenStreetMap
    gpkg_path = f"../../ltnConnect_external/exports/{placeinfo['nominatimstring']}_centroids.gpkg"

    # Read POIs from the local GeoPackage
    try:
        gdf = gpd.read_file(gpkg_path)
        print(gdf.crs)

        # Now snap to closest nodes in the street network and save the nearest node ids
        nnids = set()
        for g in gdf['geometry']:
            # Find nearest node in the carall network
            n = ox.distance.nearest_nodes(G_carall, g.x, g.y)
            if n not in nnids and haversine((g.y, g.x), (G_carall.nodes[n]["y"], G_carall.nodes[n]["x"]), unit="m") <= snapthreshold:
                nnids.add(n)
        nnids_csv_path = PATH["data"] + placeid + "/" + placeid + '_poi_' + 'neighbourhoods' + '_nnidscarall.csv'
        with open(nnids_csv_path, 'w') as f:
            for item in nnids:
                f.write("%s\n" % item)

        gdf = gdf.apply(lambda c: c.astype(str) if c.name != 'geometry' else c, axis=0)
        gpkg_output_path = PATH["data"] + placeid + "/" + placeid + '_poi_' + 'neighbourhoods' + '.gpkg'
        try:
            gdf.to_file(gpkg_output_path, driver='GPKG')
        except Exception as e:
            print(f"Notice: Writing the gdf did not work for {placeid}. Error: {e}")

        # Optionally plot for debugging purposes
        if debug:
            gdf.plot(color='red')

    except Exception as e:
        print(f"No Centroids created for {placeinfo['name']}. No POIs created. Error: {e}")
 """

### Joined Neighbourhoods and tesselation

In [ ]:
""" for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):
    print(f"{placeid}: Processing tessellation and neighbourhood points")
    location = locations[placeid]

    # Get urban areas for seed points
    urban_areas = get_urban_areas(placeinfo["nominatimstring"])
    tess = Tessellation(urban_areas)
    tess.get_polygon()  # Optional plot for debugging
    hexagons = tess.hexagons(resolution=8)

    # Load neighbourhoods and create GeoDataFrame
    neighbourhoods = load_neighbourhoods(PATH["data"] + placeid + "/")
    neighbourhood_geoms = [geom for gdf in neighbourhoods.values() for geom in gdf['geometry'].values]
    neighbourhoods_gdf = gpd.GeoDataFrame(geometry=neighbourhood_geoms, crs="EPSG:4326")

    # Extract neighbourhood centroids
    all_centroids = gpd.GeoDataFrame(columns=['source', 'geometry'], crs='EPSG:4326')
    neighbourhood_centroids = gpd.GeoDataFrame(
        [{'source': 'neighbourhood', 'geometry': geom.centroid} for geom in neighbourhoods_gdf.geometry],
        crs="EPSG:4326"
    )
    all_centroids = pd.concat([all_centroids, neighbourhood_centroids], ignore_index=True)

    # Remove hexagons that intersect with neighbourhoods
    hexagons = hexagons[~hexagons.intersects(neighbourhoods_gdf.unary_union)]
    hexagon_centroids_gdf = gpd.GeoDataFrame(geometry=hexagons.centroid, crs=hexagons.crs)

    # Add tessellation centroids to the combined GeoDataFrame
    tessellation_centroids = gpd.GeoDataFrame(
        [{'source': 'tessellation', 'geometry': g} for g in hexagon_centroids_gdf['geometry']],
        crs="EPSG:4326"
    )
    all_centroids = pd.concat([all_centroids, tessellation_centroids], ignore_index=True)

    # Snap points to the carall network
    G_carall = G_caralls_simplified[placeid]
    nnids_carall = set()
    snapped_points_carall = []

    for g in all_centroids['geometry']:
        n = ox.distance.nearest_nodes(G_carall, g.x, g.y)
        distance_to_node = haversine((g.y, g.x), (G_carall.nodes[n]["y"], G_carall.nodes[n]["x"]), unit="m")
        if distance_to_node <= snapthreshold:
            snapped_points_carall.append({'geometry': Point(G_carall.nodes[n]["x"], G_carall.nodes[n]["y"]), 'source': 'snapped'})
            nnids_carall.add(n)
        else:
            snapped_points_carall.append({'geometry': g, 'source': 'unsnapped'})

    snapped_gdf_carall = gpd.GeoDataFrame(snapped_points_carall, crs=all_centroids.crs)

    # Save nnids to CSV
    nnids_csv_path = PATH["data"] + placeid + "/" + placeid + '_poi_' + 'tess_hoods_' + 'nnidscarall' + '.csv'
    with open(nnids_csv_path, 'w') as f:
        for item in nnids_carall:
            f.write(f"{item}\n")

    # Debug plot
    if debug:
        fig, ax = plt.subplots(figsize=(12, 12))
        hexagons.plot(ax=ax, color="lightgray", edgecolor="white", alpha=0.5, label="Hexagons")
        snapped_gdf_carall[snapped_gdf_carall['source'] == 'snapped'].plot(
            ax=ax, color='green', marker='o', label='Snapped Points', markersize=5
        )
        snapped_gdf_carall[snapped_gdf_carall['source'] == 'unsnapped'].plot(
            ax=ax, color='red', marker='x', label='Unsnapped Points', markersize=5
        )
        plt.legend()
        plt.title(f"Snapped Points for {placeid}")
        plt.show()

    # Save the combined points to GeoPackage
    gpkg_output_path = PATH["data"] + placeid + "/" + placeid + '_poi_' + 'tess_hoods' + '.gpkg'
    try:
        snapped_gdf_carall.to_file(gpkg_output_path, driver="GPKG")
    except Exception as e:
        print(f"Failed to save snapped GeoDataFrame for {placeid}. Error: {e}")
 """

### Railway stations/halts and bus stops

In [ ]:
# # Here POIs are downloaded and matched to the network. To ensure consistent reproducibility, we should download POIs only once, then load them locally. For now we leave it as is, as POIs are not expected to change fast.

# for placeid, placeinfo in tqdm(cities.items(), desc = "Cities"):
#     print(placeid + ": Creating POIs")
    
#     # We need the carall graph and location geometry
#     location = locations[placeid]
#     G_carall = G_caralls_simplified[placeid]
    
#     for poiid, poitag in poiparameters.items():
#         try:
#             gdf = ox.geometries.geometries_from_polygon(location, poitag)
#             gdf = gdf[gdf['geometry'].type == "Point"] # only consider points, no polygons etc
#             # Now snap to closest nodes in street network, save the nearest node ids
#             nnids = set()
#             for g in gdf['geometry']:
#                 n = ox.distance.nearest_nodes(G_carall, g.x, g.y) # !! get_nearest_node() had [y(lat),x(lon)], now nearest_nodes() has x(lon),y(lat)
#                 if not n in nnids and haversine((g.y, g.x), (G_carall.nodes[n]["y"], G_carall.nodes[n]["x"]), unit="m") <= snapthreshold:
#                     nnids.add(n)
#             with open(PATH["data"] + placeid + "/" + placeid + '_' + 'poi_' + poiid + '_nnidscarall.csv', 'w') as f:
#                 for item in nnids:
#                     f.write("%s\n" % item)
    
#             gdf = gdf.apply(lambda c: c.astype(str) if c.name != 'geometry' else c, axis=0)
#             try: # For some cities writing the gdf does not work (i.e. London, Manhattan)
#                 gdf.to_file(PATH["data"] + placeid + "/" + placeid + '_' + 'poi_' + poiid + '.gpkg', driver = 'GPKG')
#             except:
#                 print("Notice: Writing the gdf did not work for " + placeid)
#             if debug: gdf.plot(color = 'red')
#         except:
#             print("No stations in " + placeinfo["name"] + ". No POIs created.")

### Grid

Using code from: https://github.com/gboeing/osmnx-examples/blob/v0.11/notebooks/17-street-network-orientations.ipynb

In [ ]:
# for placeid, placeinfo in tqdm(cities.items(), desc  = "Cities"):
#     print(placeid + ": Creating grid")
    
#     location = locations[placeid]
    
    
    
#     # FIRST, determine the most common bearing, for the best grid orientation
#     G = G_caralls[placeid]
#     bearings = {}    
#     # calculate edge bearings
#     Gu = ox.bearing.add_edge_bearings(ox.get_undirected(G))

#     # weight bearings by length (meters)
#     city_bearings = []
#     for u, v, k, d in Gu.edges(keys = True, data = True):
#         city_bearings.extend([d['bearing']] * int(d['length']))
#     b = pd.Series(city_bearings)
#     bearings = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop = 'True')

#     bins = np.arange(bearingbins + 1) * 360 / bearingbins
#     count = count_and_merge(bearingbins, bearings)
#     principalbearing = bins[np.where(count == max(count))][0]
#     if debug: 
#         print("Principal bearing: " + str(principalbearing))


#     # SECOND, construct the grid
#     G = G_caralls_simplified[placeid]

#     # 1) Get lat lon window, with buffer for snapping outside POIs
#     # https://stackoverflow.com/questions/7477003/calculating-new-longitude-latitude-from-old-n-meters
#     buf = max(((2*snapthreshold) / 6378000) * (180 / math.pi), 
#               ((2*snapthreshold) / 6378000) * (180 / math.pi) / math.cos(location.centroid.y * math.pi/180)
#              )
#     cities[placeid]["bbox"] = location.buffer(buf).bounds

#     # 2) Generate abstract grid points in window
#     # https://stackoverflow.com/questions/40342355/how-can-i-generate-a-regular-geographic-grid-using-python
#     # Set up projections
#     p_ll = pyproj.Proj('+proj=longlat +datum=WGS84')
#     aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
#     p_mt = pyproj.Proj(aeqd_proj.format(lat=location.centroid.y, lon=location.centroid.x)) # equidistant projection

#     # Make the area larger to account for tilted grid
#     deltax = cities[placeid]["bbox"][2] - cities[placeid]["bbox"][0]
#     deltay = cities[placeid]["bbox"][3] - cities[placeid]["bbox"][1]
#     enlargefactor = 10

#     # Create corners of rectangle to be transformed to a grid
#     sw = shapely.geometry.Point((cities[placeid]["bbox"][0], cities[placeid]["bbox"][1]))
#     ne = shapely.geometry.Point((cities[placeid]["bbox"][2]+enlargefactor*deltax, cities[placeid]["bbox"][3]+enlargefactor*deltay))
    
#     # Project corners to target projection
#     transformed_sw = pyproj.transform(p_ll, p_mt, sw.x, sw.y) # Transform NW point to equidistant
#     transformed_ne = pyproj.transform(p_ll, p_mt, ne.x, ne.y) # .. same for SE

#     # Iterate over 2D area
#     principalbearing = principalbearing % 90 # Take it modulo 90 because it will be a square grid
#     if principalbearing > 45:
#         principalbearing -= 90 # Make bearing fall btw -45 and 45

#     xcoords = np.arange(transformed_sw[0], transformed_ne[0], gridl)
#     ycoords = np.arange(transformed_sw[1], transformed_ne[1], gridl)
#     xsize =  xcoords.size
#     ysize = ycoords.size
#     xcoords = np.tile(xcoords, ysize)
#     ycoords = np.repeat(ycoords, xsize)
#     gridpoints=[(x, y) for x, y in zip(xcoords, ycoords)]
#     new_points = rotate_grid(gridpoints, origin = transformed_sw, degrees = principalbearing)
    
#     # https://stackoverflow.com/questions/42459068/projecting-a-numpy-array-of-coordinates-using-pyproj
#     fx, fy = pyproj.transform(p_mt, p_ll, new_points[:,0], new_points[:,1])
#     gridpoints = np.dstack([fx, fy])[0]
#     if principalbearing >=0:
#         # If we rotated right, we need to shift everything to the left
#         gridpoints[:,0] -= 0.4*enlargefactor*deltax*math.sin(np.deg2rad(principalbearing))
#     else:
#         # If we rotated left, we need to shift everything down and to the right
#         gridpoints[:,0] += 0.4*enlargefactor*deltax*math.sin(np.deg2rad(principalbearing))
#         gridpoints[:,1] -= 0.4*enlargefactor*deltay

#     # Cut back to bounding box
#     mask = (gridpoints[:,0] >= cities[placeid]["bbox"][0]) & (gridpoints[:,0] <= cities[placeid]["bbox"][2]) & (gridpoints[:,1] >= cities[placeid]["bbox"][1]) & (gridpoints[:,1] <= cities[placeid]["bbox"][3])
#     gridpoints_cut = gridpoints[mask]
    
#     if debug:
#         fig = plt.figure(figsize=[2*6.4, 2*4.8])
#         ax = fig.add_axes([0, 0, 1, 1])
#         ax.set_aspect('equal', adjustable = 'datalim')
#         plt.plot([g[0] for g in gridpoints_cut], [g[1] for g in gridpoints_cut], ".", color = "red")

#     # 3) Snap grid points to map
#     nnids = set()
#     for g in gridpoints_cut:
#         n = ox.distance.nearest_nodes(G, g[0], g[1]) # !! get_nearest_node() had [y(lat),x(lon)], now nearest_nodes() has x(lon),y(lat)
#         if n not in nnids and haversine((g[1], g[0]), (G.nodes[n]["y"], G.nodes[n]["x"]), unit="m") <= snapthreshold:
#             nnids.add(n)
#     with open(PATH["data"] + placeid + "/" + placeid + '_poi_grid_nnidscarall.csv', 'w') as f:
#         for item in nnids:
#             f.write("%s\n" % item)
    
#     # 4) Save grid points to geopackage
#     geometry = [Point(xy) for xy in gridpoints_cut]  # Convert each point to a Shapely Point object
#     gdf = gpd.GeoDataFrame(geometry=geometry, crs="EPSG:4326")  # Create GeoDataFrame, assuming EPSG:4326 for WGS84
    
#     # Ensure all columns except geometry are strings for consistency with your POI export
#     gdf = gdf.apply(lambda c: c.astype(str) if c.name != 'geometry' else c, axis=0)

#     # Try writing the grid points to a GeoPackage
#     try:
#         gdf.to_file(PATH["data"] + placeid + "/" + placeid + '_poi_grid.gpkg', driver='GPKG')
#         print(f"Grid points written to GeoPackage for {placeid}")
#     except Exception as e:
#         print(f"Notice: Writing the grid points to GeoPackage did not work for {placeid}: {str(e)}")

#     if debug:
#         gdf.plot(color='blue')

### Population density (for now commented out as unfinished: data not reliable enough)

In [ ]:
# # https://medium.com/@danielmoyo/raster-data-in-python-part-i-reading-and-visualising-raster-data-95cb072fcbfa
# import rasterio

# densityfolder = "populationdensity"
# for placeid, placeinfo in cities.items():
#     raster_path = PATH["data"] + densityfolder + "/" + placeinfo['countryid'] + "_ppp_2020_UNadj.tif"
#     pop_raster = rasterio.open(raster_path)
    
#     cities[placeid]["bbox"] = location.bounds

#     def plot_raster(arr, title=None, figsize=(30,10)):
#         """ helper function to plot a raster array """
#         fig, ax = plt.subplots(figsize=figsize)
#         plt.imshow(arr)
#         plt.colorbar()
#         plt.title(f'{title}')
#         plt.xlabel('Column #')
#         plt.ylabel('Row #')
#         return fig, ax

#     pop_data = pop_raster.read(1)
#     pop_data[pop_data<0] = 0
#     pop_count = pop_data[pop_data>0].sum()
#     plot_raster(pop_data, title=pop_count)

In [ ]:
# # https://stackoverflow.com/questions/50191648/gis-geotiff-gdal-python-how-to-get-coordinates-from-pixel
# # https://gdal.org/api/gdaldataset_cpp.html#_CPPv4N11GDALDataset15GetGeoTransformEPd

# # Problems:
# # https://gis.stackexchange.com/questions/331908/notimplementederror-wrong-number-or-type-of-arguments-for-overloaded-function
# # https://gis.stackexchange.com/questions/44040/osr-python-testsr-importfromepsg4326-returns-empty-spatial-reference
# # https://www.programcreek.com/python/example/58591/osgeo.osr.SpatialReference


# def generate_latlons(raster_path):
#     # open the dataset and get the geo transform matrix
#     ds = gdal.Open(raster_path) 
#     xoffset, px_w, rot1, yoffset, px_h, rot2 = ds.GetGeoTransform()
#     ds_arr = ds.ReadAsArray()
    
#     x_arr = np.arange(0, ds_arr.shape[0], 1)
#     x_arr.shape = (len(x_arr), 1)
#     x_arr = np.repeat(x_arr, [ds_arr.shape[1]], axis=1)
    
#     y_arr = np.arange(0, ds_arr.shape[1], 1)
#     y_arr.shape = (1, len(y_arr))
#     y_arr = np.repeat(y_arr, [ds_arr.shape[0]], axis=0)
    
#     # supposing x and y are your pixel coordinate this 
#     # is how to get the coordinate in space.
#     lon_arr = xoffset + px_w * y_arr + rot1 * x_arr + px_w / 2.0
#     lat_arr = yoffset + px_h * y_arr + rot2 * x_arr + px_h / 2.0
    
#     return(lat_arr, lon_arr)

# for placeid, placeinfo in cities.items():
#     raster_path = PATH["data"] + densityfolder + "/" + placeinfo['countryid'] + "_ppp_2020_UNadj.tif"
#     pop_raster = rasterio.open(raster_path)
#     (lat, lon) = generate_latlons(raster_path)
    
# # To do: only read the relevant window
# # https://medium.com/@danielmoyo/raster-data-in-python-part-ii-working-with-geographic-coordinates-a368708de5f2

In [ ]:
# indices = np.where(pop_data == pop_data.max())
# c_x = indices[0].item()
# c_y = indices[1].item()

# delta = 60
# plot_raster(pop_data[c_x-delta:c_x+delta, c_y-delta:c_y+delta], title="Population density, Copenhagen")